# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
S. Jiao  ->  S. Jiao  |  ['S. Jiao']
F. Walter  ->  F. Walter  |  ['F. Walter']
Y. Khusanova  ->  Y. Khusanova  |  ['Y. Khusanova']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']


Arxiv has 102 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.23319


extracting tarball to tmp_2503.23319... done.
Retrieving document from  https://arxiv.org/e-print/2503.23700


extracting tarball to tmp_2503.23700...

 done.
Retrieving document from  https://arxiv.org/e-print/2503.23797


...

Retrieving document from  https://arxiv.org/e-print/2503.24280


extracting tarball to tmp_2503.24280...

 done.


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2503.24280/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: '10-appendixD' from 'tmp_2503.24280/10-appendixD.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: '9-appendixC' from 'tmp_2503.24280/9-appendixC.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/ar

Issues with the citations
list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.24280-b31b1b.svg)](https://arxiv.org/abs/2503.24280) | **BOWIE-ALIGN: Sub-stellar metallicity and carbon depletion in the aligned TrES-4b with JWST NIRSpec transmission spectroscopy**  |
|| A. Meech, et al. -- incl., <mark>E.-M. Ahrer</mark> |
|*Appeared on*| *2025-04-01*|
|*Comments*| *23 pages, 20 figures, 7 tables. Accepted to MNRAS on 26 March 2025*|
|**Abstract**|            The formation and migration history of a planet is expected to be imprinted in its atmosphere, in particular its carbon-to-oxygen (C/O) ratio and metallicity. The BOWIE-ALIGN programme is performing a comparative study of JWST spectra of four aligned and four misaligned hot Jupiters, with the aim of characterising their atmospheres and corroborating the link between the observables and the formation history. In this work, we present the $2.8-5.2$ micron transmission spectrum of TrES-4b, a hot Jupiter with an orbit aligned with the rotation axis of its F-type host star. Using free chemistry atmospheric retrievals, we report a confident detection of H$_2$O at an abundance of $\log X_\mathrm{H_2O}=-2.98^{+0.68}_{-0.73}$ at a significance of $8.4\sigma$. We also find evidence for CO and small amounts of CO$_2$, retrieving abundances $\log X_\mathrm{CO}= -3.76^{+0.89}_{-1.01}$ and $\log X_\mathrm{CO_2}= -6.86^{+0.62}_{-0.65}$ ($3.1\sigma$ and $4.0\sigma$ respectively). The observations are consistent with the the atmosphere being in chemical equilibrium; our retrievals yield $\mathrm{C/O}$ between $0.30-0.42$ and constrain the atmospheric metallicity to the range $0.4-0.7\times$ solar. The inferred sub-stellar properties (C/O and metallicity) challenge traditional models, and could have arisen from an oxygen-rich gas accretion scenario, or a combination of low-metallicity gas and carbon-poor solid accretion.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.23319-b31b1b.svg)](https://arxiv.org/abs/2503.23319) | **Class I/II Jets with JWST: Mass loss rates, Asymmetries, and Binary induced Wigglings**  |
|| N. S. Bajaj, et al. -- incl., <mark>K. Schwarz</mark>, <mark>D. Semenov</mark> |
|*Appeared on*| *2025-04-01*|
|*Comments*| *Accepted for publication in the Astronomical Journal. 15 figures and 3 tables*|
|**Abstract**|            We present JWST NIRSpec spectro-imaging observations of jets from four edge-on protoplanetary disks that exhibit clear signatures of MHD disk winds. Bipolar jets are detected and spatially resolved in over 30 shock-excited forbidden lines, multiple Paschen and Brackett series lines of atomic hydrogen, and the high-energy excitation line of atomic helium (1.083 um). This helium line is the brightest jet-tracer towards HH 30 and FS TauB, which also exhibit asymmetric intensity between their red- and blue-shifted lobes in all tracers, including the [Fe II] and [He I] lines. Extinction maps reveal no significant differences across the lobes, suggesting an asymmetric jet-launching mechanism rather than environmental effects. Diagnostic line ratios yield consistent shock speeds of 50-60 km/s, jet ionization fractions of 0.1-0.2, and pre-shock electron densities of 1000 /cm^3. Combined with pixel-by-pixel electron density maps and [Fe II] line luminosities, we estimate jet mass-loss rates using three independent methods, averaging around a few 10^(-9) solar masses/yr. We estimate the accretion rates for these sources as 10 times the jet mass loss rates and find them to match well with the independently derived accretion estimates of other Class II sources in the Taurus star-forming region. Owing to JWST's high precision, we also investigate jet wiggling and find Tau 042021 to showcase the perfect case of mirror-symmetric wiggling, which can only be explained by the motion of the jet source around a stellar companion. Modeling this wiggling suggests Tau 042021 to host 0.33 and 0.07 solar masses binary at the center with binary separation of 1.35 au and an orbital period of 2.5 years.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.23700-b31b1b.svg)](https://arxiv.org/abs/2503.23700) | **Dual-band Unified Exploration of Three CMZ Clouds (DUET). Cloud-wide census of continuum sources showing low spectral indices**  |
|| F. Xu, et al. -- incl., <mark>S. Jiao</mark> |
|*Appeared on*| *2025-04-01*|
|*Comments*| *16 pages, 10 figures. Accepted for publication in A&A. For interactive data visualization, see this https URL. The continuum images will be available once the paper is published*|
|**Abstract**|            The Milky Way's Central Molecular Zone (CMZ) is measured to form stars 10 times less efficiently than in the Galactic disk, based on emission from high-mass stars. However, the CMZ's low-mass protostellar population, which accounts for most of the initial stellar mass budget and star formation rate (SFR), is poorly constrained observationally due to limited sensitivity and resolution. We present the Dual-band Unified Exploration of Three CMZ Clouds (DUET) survey, targeting the 20 km/s Cloud, Sgr C, and Dust Ridge cloud e using the Atacama Large Millimeter/submillimeter Array (ALMA) at 1.3 and 3 mm. The mosaicked observations achieve a comparable resolution of 0.2-0.3" (~1600-2500 au) and a sky coverage of 8.3-10.4 square arcmin, respectively. We report 563 continuum sources at 1.3 mm and 330 at 3 mm, respectively, and a dual-band catalog with 450 continuum sources. These sources are marginally resolved at the 2,000 au resolution. We find a cloud-wide deviation (>70%) from commonly-used dust modified blackbody (MBB) models, characterized by either low spectral indices or low brightness temperatures. Three possible explanations for the deviation are discussed. (1) Optically thick Class 0/I Young stellar objects (YSOs) with very small beam filling factors can lead to lower brightness temperatures than what MBB models predict. (2) Large (mm/cm-sized) dust grains have more significant self-scattering, and therefore frequency-dependent albedo could cause lower spectral indices. (3) Free-free emission over 30 uJy can severely contaminate dust emission and cause low spectral indices for mJy sources in our sample, although the needed number of massive protostars (embedded UCHII regions) is infeasibly high for the normal stellar initial mass function. A reliable measurement of the SFR at low protostellar masses will require future work to distinguish between these possible explanations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.23797-b31b1b.svg)](https://arxiv.org/abs/2503.23797) | **Kiloparsec-Scale Alignment of a Radio Jet with Cool Gas and Dust in a z~6 Quasar**  |
|| <mark>F. Walter</mark>, et al. -- incl., <mark>Y. Khusanova</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2025-04-01*|
|*Comments*| *Accepted for publication in the ApJ Letters*|
|**Abstract**|            We present high-angular resolution (0.068", ~400pc) ALMA imaging of the [CII] line and dust continuum emission of PSO J352.4034-15.3373, a radio-loud quasar at z=5.83. The observations reveal a remarkably close match between the orientation of the [CII] and thermal dust emission mapped by ALMA, and radio synchrotron emission of a radio jet previously mapped by the VLBA. This narrow alignment extends over ~4kpc, reminiscent of the well-studied 'alignment effect' in lower-redshift radio galaxies. The [CII] kinematics show a linear increase in velocity with galactocentric radii up to ~200 km/s at r=2kpc, consistent with bulk motions within the galaxy potential, and not relativistic jet motions. The kinematics and respective morphologies are consistent with a picture in which the relativistic jet injects energy into the interstellar medium (potentially leading to subsequent star formation), giving rise to the observed alignment and significant (> 100 km/s) [CII] velocity dispersion within the host galaxy on kiloparsec scales. Indeed, the astonishingly close alignment and narrow linearity of the radio jet with the [CII] and dust emission are hard to conceive without some fundamental relationship between the two.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error string index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.24280/./transmission_spectra_red.png', 'tmp_2503.24280/./transmission_spectra_diffres_red.png', 'tmp_2503.24280/./tres-4b_retrieved_PT_madhu.png']
copying  tmp_2503.24280/./transmission_spectra_red.png to _build/html/
copying  tmp_2503.24280/./transmission_spectra_diffres_red.png to _build/html/
copying  tmp_2503.24280/./tres-4b_retrieved_PT_madhu.png to _build/html/
exported in  _build/html/2503.24280.md
    + _build/html/tmp_2503.24280/./transmission_spectra_red.png
    + _build/html/tmp_2503.24280/./transmission_spectra_diffres_red.png
    + _build/html/tmp_2503.24280/./tres-4b_retrieved_PT_madhu.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\mystar}{TrES-4\xspace}$
$\newcommand{\planet}{TrES-4 b\xspace}$
$\newcommand{\tiberius}[1]{\texttt{Tiberius}\xspace}$
$\newcommand{\eureka}[1]{\texttt{Eureka!}\xspace}$
$\newcommand{\poseidon}[1]{\texttt{POSEIDON}\xspace}$
$\newcommand{\prt}{\texttt{petitRADTRANS}\xspace}$
$\newcommand{\redfootnote}[1]{\textcolor{red}{\footnote{\textcolor{red}{#1}}}}$
$\newcommand{\vdag}{\dagger}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand\mn{@urlcharsother}$
$\newcommand\mn{@doi}$
$\newcommand\mn{@doi@}$
$\newcommand\mn{@eprint#1#2}$
$\newcommand\mn{@eprint@arXiv#1}$
$\newcommand\mn{@eprint@dblp#1}$
$\newcommand\mn{@eprint@#1:#2:#3:#4}$
$\newcommand{\@}{tempa}$
$\newcommand{\@}{tempa }$
$\newcommand{\@}{tempb }$
$\newcommand{\@}{tempc }$
$\newcommand{\@}{tempb }$</div>



<div id="title">

# BOWIE-ALIGN: Sub-stellar metallicity and carbon depletion in the aligned TrES-4b with JWST NIRSpec transmission spectroscopy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.24280-b31b1b.svg)](https://arxiv.org/abs/2503.24280)<mark>Appeared on: 2025-04-01</mark> -  _23 pages, 20 figures, 7 tables. Accepted to MNRAS on 26 March 2025_

</div>
<div id="authors">

A. Meech, et al. -- incl., <mark>E.-M. Ahrer</mark>

</div>
<div id="abstract">

**Abstract:** The formation and migration history of a planet is expected to be imprinted in its atmosphere, in particular its carbon-to-oxygen (C/O) ratio and metallicity.The BOWIE-ALIGN programme is performing a comparative study of JWST spectra of four aligned and four misaligned hot Jupiters, with the aim of characterising their atmospheres and corroborating the link between the observables and the formation history.In this work, we present the $2.8-5.2$ micron transmission spectrum of $\planet$ , a hot Jupiter with an orbit aligned with the rotation axis of its F-type host star.Using free chemistry atmospheric retrievals, we report a confident detection of $H_2$ O at an abundance of $\log X_\mathrm{H_2O}=-2.98^{+0.68}_{-0.73}$ at a significance of $8.4\sigma$ .We also find evidence for CO and small amounts of $CO_2$ , retrieving abundances $\log X_\mathrm{CO}= -3.76^{+0.89}_{-1.01}$ and $\log X_\mathrm{CO_2}= -6.86^{+0.62}_{-0.65}$ ( $3.1\sigma$ and $4.0\sigma$ respectively).The observations are consistent with the the atmosphere being in chemical equilibrium; our retrievals yield $\mathrm{C/O}$ between $0.30-0.42$ and constrain the atmospheric metallicity to the range $0.4-0.7\times$ solar.The inferred sub-stellar properties (C/O and metallicity) challenge traditional models, and could have arisen from an oxygen-rich gas accretion scenario, or a combination of low-metallicity gas and carbon-poor solid accretion.

</div>

<div id="div_fig1">

<img src="tmp_2503.24280/./transmission_spectra_red.png" alt="Fig2" width="100%"/>

**Figure 2. -** **Top panel:** The $R\simeq100$ transmission spectrum of $\planet$ from the \texttt{Tiberius} and \texttt{Eureka!} reductions in pink and blue respectively. **Middle panel:** The precision of each spectrum and **bottom panel:** the difference between the two reductions. The $1/2\sigma$ intervals are shaded for reference. (*fig:transmission-spec*)

</div>
<div id="div_fig2">

<img src="tmp_2503.24280/./transmission_spectra_diffres_red.png" alt="Fig9" width="100%"/>

**Figure 9. -** The $R\simeq100$ and $R\simeq400$ transmission spectra from the three independent reductions using **(top panel):**$\tiberius$**(middle):**$\eureka$  and **(bottom):**$\tiberius$ second reduction as detailed in Appendix \ref{SECTION-7:appendix-tiberiusJK}. (*fig-app:R400*)

</div>
<div id="div_fig3">

<img src="tmp_2503.24280/./tres-4b_retrieved_PT_madhu.png" alt="Fig10" width="100%"/>

**Figure 10. -** The median retrieved PT profile (mean and $1/2\sigma$ regions in grey) of the chemical equilibrium reference retrieval on the $\tiberius$$R\simeq100$ transmission spectrum, having implemented the more complex PT profile of \citet{Madhusudhan2009}.
    Overplotted in pink is the median retrieved gradient PT profile on the same spectrum (preferred), and in purple is the photometric contribution function across the full NIRSpec G395H bandpass. (*fig-app:madhu-PT*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.24280"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

418  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

16  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
